# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Sep 29 2022  8:30AM  247519        10   WT5357 Out     Eywa Pharma Inc.   
1  Sep 28 2022  3:28PM  247515        19  ADV80006020  AdvaGen Pharma, Ltd   
2  Sep 28 2022  3:21PM  247513        19  ADV80006046  AdvaGen Pharma, Ltd   
3  Sep 28 2022  3:21PM  247513        19  ADV80006047  AdvaGen Pharma, Ltd   
4  Sep 28 2022  3:21PM  247513        19  ADV80006048  AdvaGen Pharma, Ltd   
5  Sep 28 2022  3:21PM  247513        19  ADV80006049  AdvaGen Pharma, Ltd   
6  Sep 28 2022  3:21PM  247513        19  ADV80006050  AdvaGen Pharma, Ltd   
7  Sep 28 2022  3:21PM  247513        19  ADV80006051  AdvaGen Pharma, Ltd   
8  Sep 28 2022  3:21PM  247513        19  ADV80006052  AdvaGen Pharma, Ltd   
9  Sep 28 2022  3:21PM  247513        19  ADV80006054  AdvaGen Pharma, Ltd   

  ShipmentStatus  
0       Released  
1       Released  
2      Executing  
3      Executing  
4      Executing  
5      Executing  
6      Executing  
7       Released  
8      Executing  
9      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
13  247511      Executing          1
14  247513      Executing         13
15  247513       Released         13
16  247515       Released          1
17  247519       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
247509                2.0        8.0       1.0
247511                NaN        1.0       NaN
247513                NaN       13.0      13.0
247515                NaN        NaN       1.0
247519                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
247460                0.0       21.0       0.0
247471                0.0        0.0       1.0
247475                0.0        2.0       0.0
247477                3.0       12.0       5.0
247480                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
247460                  0         21         0
247471                  0          0         1
247475                  0          2         0
247477                  3         12         5
247480                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               247460          0         21         0
1               247471          0          0         1
2               247475          0          2         0
3               247477          3         12         5
4               247480          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               247460                  21         
1               247471                            1
2               247475                   2         
3               247477         3        12        5
4               247480                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Sep 29 2022  8:30AM  247519        10                Eywa Pharma Inc.
1   Sep 28 2022  3:28PM  247515        19             AdvaGen Pharma, Ltd
2   Sep 28 2022  3:21PM  247513        19             AdvaGen Pharma, Ltd
28  Sep 28 2022  2:52PM  247511        19           ACG North America LLC
29  Sep 28 2022  1:53PM  247509        10               ISDIN Corporation
40  Sep 28 2022 12:11PM  247494        22               NBTY Global, Inc.
41  Sep 28 2022 12:11PM  247493        22               NBTY Global, Inc.
42  Sep 28 2022 12:09PM  247492        22               NBTY Global, Inc.
43  Sep 28 2022 11:02AM  247480        16  Sartorius Bioprocess Solutions
44  Sep 28 2022 10:57AM  247475        19          GCH Granules USA, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Sep 29 2022  8:30AM  247519        10                Eywa Pharma Inc.   
1  Sep 28 2022  3:28PM  247515        19             AdvaGen Pharma, Ltd   
2  Sep 28 2022  3:21PM  247513        19             AdvaGen Pharma, Ltd   
3  Sep 28 2022  2:52PM  247511        19           ACG North America LLC   
4  Sep 28 2022  1:53PM  247509        10               ISDIN Corporation   
5  Sep 28 2022 12:11PM  247494        22               NBTY Global, Inc.   
6  Sep 28 2022 12:11PM  247493        22               NBTY Global, Inc.   
7  Sep 28 2022 12:09PM  247492        22               NBTY Global, Inc.   
8  Sep 28 2022 11:02AM  247480        16  Sartorius Bioprocess Solutions   
9  Sep 28 2022 10:57AM  247475        19          GCH Granules USA, Inc.   

  Completed Executing Released  
0                            1  
1                            1  
2                  13       13  
3                   1           
4         2         8        1  
5                            1  
6                            1  
7                            1  
8                            1  
9                   2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Sep 29 2022  8:30AM  247519        10                Eywa Pharma Inc.   
1  Sep 28 2022  3:28PM  247515        19             AdvaGen Pharma, Ltd   
2  Sep 28 2022  3:21PM  247513        19             AdvaGen Pharma, Ltd   
3  Sep 28 2022  2:52PM  247511        19           ACG North America LLC   
4  Sep 28 2022  1:53PM  247509        10               ISDIN Corporation   
5  Sep 28 2022 12:11PM  247494        22               NBTY Global, Inc.   
6  Sep 28 2022 12:11PM  247493        22               NBTY Global, Inc.   
7  Sep 28 2022 12:09PM  247492        22               NBTY Global, Inc.   
8  Sep 28 2022 11:02AM  247480        16  Sartorius Bioprocess Solutions   
9  Sep 28 2022 10:57AM  247475        19          GCH Granules USA, Inc.   

  Released Executing Completed  
0        1                      
1        1                      
2       13        13            
3                  1            
4        1         8         2  
5        1                      
6        1                      
7        1                      
8        1                      
9                  2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Sep 29 2022  8:30AM  247519        10       Eywa Pharma Inc.        1   
1  Sep 28 2022  3:28PM  247515        19    AdvaGen Pharma, Ltd        1   
2  Sep 28 2022  3:21PM  247513        19    AdvaGen Pharma, Ltd       13   
3  Sep 28 2022  2:52PM  247511        19  ACG North America LLC            
4  Sep 28 2022  1:53PM  247509        10      ISDIN Corporation        1   

  Executing Completed  
0                      
1                      
2        13            
3         1            
4         8         2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Sep 29 2022  8:30AM  247519        10       Eywa Pharma Inc.       1.0   
1  Sep 28 2022  3:28PM  247515        19    AdvaGen Pharma, Ltd       1.0   
2  Sep 28 2022  3:21PM  247513        19    AdvaGen Pharma, Ltd      13.0   
3  Sep 28 2022  2:52PM  247511        19  ACG North America LLC       NaN   
4  Sep 28 2022  1:53PM  247509        10      ISDIN Corporation       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2       13.0        NaN  
3        1.0        NaN  
4        8.0        2.0

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Sep 29 2022  8:30AM  247519        10       Eywa Pharma Inc.       1.0   
1  Sep 28 2022  3:28PM  247515        19    AdvaGen Pharma, Ltd       1.0   
2  Sep 28 2022  3:21PM  247513        19    AdvaGen Pharma, Ltd      13.0   
3  Sep 28 2022  2:52PM  247511        19  ACG North America LLC       0.0   
4  Sep 28 2022  1:53PM  247509        10      ISDIN Corporation       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2       13.0        0.0  
3        1.0        0.0  
4        8.0        2.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                        
10         989976       8.0       20.0        5.0
15         247460       0.0       21.0        0.0
16         247480       1.0        0.0        0.0
19         990014      14.0       16.0        0.0
22         742479       3.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse      id  Released  Executing  Completed
0        10  989976       8.0       20.0        5.0
1        15  247460       0.0       21.0        0.0
2        16  247480       1.0        0.0        0.0
3        19  990014      14.0       16.0        0.0
4        22  742479       3.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10       8.0       20.0        5.0
1        15       0.0       21.0        0.0
2        16       1.0        0.0        0.0
3        19      14.0       16.0        0.0
4        22       3.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released    8.0
1        15  Released    0.0
2        16  Released    1.0
3        19  Released   14.0
4        22  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16    19   22
Status                               
Completed   5.0   0.0  0.0   0.0  0.0
Executing  20.0  21.0  0.0  16.0  0.0
Released    8.0   0.0  1.0  14.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16    19   22
0          Completed   5.0   0.0  0.0   0.0  0.0
1          Executing  20.0  21.0  0.0  16.0  0.0
2           Released   8.0   0.0  1.0  14.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16    19   22
0  Completed   5.0   0.0  0.0   0.0  0.0
1  Executing  20.0  21.0  0.0  16.0  0.0
2   Released   8.0   0.0  1.0  14.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()